In [1]:
import os
import json
import requests
from datetime import datetime, timezone

BASE_URL = "https://api.qtcq.xyz"

def get_symbols(base_url=BASE_URL, timeout=10):
    url = f"{base_url}/api/v1/market-data/symbols"
    r = requests.get(url, timeout=timeout)
    r.raise_for_status()
    return r.json()

def get_data_repair_status(base_url=BASE_URL, timeout=10):
    url = f"{base_url}/api/v1/data-repair/status"
    r = requests.get(url, timeout=timeout)
    r.raise_for_status()
    return r.json()

def pretty_print_symbols(resp):
    symbols = resp.get("symbols", [])
    equity = resp.get("equity_symbols", [])
    count = resp.get("count", len(symbols))

    print("\n=== QTC Symbols Summary ===")
    print(f"Total symbols (count): {count}")
    print(f"Equity symbols: {len(equity)}")
    print(f"Non-equity symbols: {count - len(equity)}")
    print("\nFirst 50 symbols:")
    print(", ".join(symbols[:50]))

def pretty_print_repair(resp):
    print("\n=== Data Repair Status ===")
    # 这些字段在手册示例里出现过；如果实际返回不同，也不会报错
    print(f"running: {resp.get('running')}")
    print(f"market_hours: {resp.get('market_hours')}")
    print(f"symbols_tracked: {resp.get('symbols_tracked')}")
    print(f"last_repair_time: {resp.get('last_repair_time')}")
    print(f"next_repair_in_minutes: {resp.get('next_repair_in_minutes')}")
    print(f"root_path: {resp.get('root_path')}")

if __name__ == "__main__":
    # 1) 查 symbols
    symbols_resp = get_symbols()
    pretty_print_symbols(symbols_resp)

    # 2) 查 data repair / symbols_tracked
    repair_resp = get_data_repair_status()
    pretty_print_repair(repair_resp)

    # 3) 可选：保存原始 JSON（方便你给团队共享）
    ts = datetime.now(timezone.utc).strftime("%Y%m%d_%H%M%S")
    with open(f"qtc_symbols_{ts}.json", "w", encoding="utf-8") as f:
        json.dump(symbols_resp, f, ensure_ascii=False, indent=2)

    with open(f"qtc_data_repair_{ts}.json", "w", encoding="utf-8") as f:
        json.dump(repair_resp, f, ensure_ascii=False, indent=2)

    print("\nSaved JSON files.")



=== QTC Symbols Summary ===
Total symbols (count): 534
Equity symbols: 532
Non-equity symbols: 2

First 50 symbols:
A, AAPL, ABBV, ABNB, ABT, ACGL, ACN, ADBE, ADI, ADM, ADP, ADSK, AEE, AEP, AES, AFL, AIG, AIZ, AJG, AKAM, ALB, ALGN, ALL, ALLE, AMAT, AMCR, AMD, AME, AMGN, AMP, AMT, AMZN, ANET, AON, AOS, APA, APD, APH, APO, APP, APTV, ARE, ATO, AVB, AVGO, AVY, AWK, AXON, AXP, AZO


HTTPError: 404 Client Error: Not Found for url: https://api.qtcq.xyz/api/v1/data-repair/status

In [2]:
import requests
import pandas as pd

BASE = "https://api.qtcq.xyz"

def get_recent(symbol="AAPL", n=200):
    url = f"{BASE}/api/v1/market-data/{symbol}/recent/{n}"
    r = requests.get(url, timeout=10)
    r.raise_for_status()
    js = r.json()
    df = pd.DataFrame(js.get("data", []))
    if not df.empty and "timestamp" in df.columns:
        df["timestamp"] = pd.to_datetime(df["timestamp"], utc=True)
        df = df.sort_values("timestamp")
    return df

df = get_recent("AAPL", 200)
print(df.tail())


HTTPError: 404 Client Error: Not Found for url: https://api.qtcq.xyz/api/v1/market-data/AAPL/recent/200